### Imports

In [191]:
import numpy as np
import sympy as sym
from numpy.random import randn
np.set_printoptions(threshold=np.inf)


### Values

In [192]:
m = 5
sigma = 2
sigma2 = 2*sigma**2
w = randn(m) / sigma
b = np.random.rand(m)*2*np.pi

n = 5
x = randn(n)/2
z = randn(n)/2 # function y

y = np.array([np.cos(x), np.sin(x)])
x_vector = np.array([randn(n)/2, randn(n)/2])

lam = 0.000001

### Gaussian Kernel

In [193]:
def k_gauss(x,z):
    """R^n x R^n -> R - should be one number
    # vector is multi-dimensional

    Args:
        x (float): real value numbers
        z (float): real value numbers

    Returns:
        (vector with length n): a scalar shift-invariant Gaussian kernel
    """
    return np.exp(-((np.linalg.norm(x-z))**2)/(sigma2)) 

print(k_gauss(x,z))

0.699255540235182


###  Scalar kernel

In [194]:
def k_approx(k, x, z):
    """scalar kernel

    Args:
        k (kernel): to test with different kernels

    Returns:
        approximated kernel: sum of feature vectors
    """
    n = len(x)
    Kxz = np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            Kxz[i, j] = k(x[i], z[j])
    return Kxz

### Approximation 1

In [195]:
def psi_approx1(x):
   """
   Args:
       x (vector): vector with length n

   Returns:
       vector: vector with length n
   """
   return np.sqrt(2)*np.cos(w*x + b)

print(psi_approx1(x))

[-1.28936631  1.08525407  0.68193362  0.56961972  1.27841529]


### Approximation 2

In [196]:
def phi_cos(x, w):
    return 1/np.sqrt(m) * np.cos(w.T@x)

def phi_sin(x, w):
    return 1/np.sqrt(m) * np.sin(w.T@x)

def add_phi_cos_sin(f, g, x):
    """
        [1 2 3] and [1 2 3] -> [1 1 2 2 3 3]
    """
    new_list = []
    for i in range (m):
        new_list.append(f(x))
        new_list.append(g(x))
    return new_list
"""
print("cos:", phi_cos(x))
print("sin:", phi_sin(x))
print("\n added together:", add_phi_cos_sin(phi_cos, phi_sin, x))
print("w:", w, w.T)
print("x:", x)
"""
def K_approx2(x, z, w):
    """The scalar kernel

    Args:
        x (float): random generated numbers
        z (float): random generated numbers

    Returns:
        (float): a number, which is the scalar kernel
    """
    return np.sum(phi_cos(x, w) * phi_cos(z, w)) + np.sum(phi_sin(x, w) * phi_sin(z, w))

"""
print(K_approx2(x,z))
"""

'\nprint(K_approx2(x,z))\n'

### Matrix kernels

In [197]:
# Matrix kernel
def K(k, *args): # *args: can give as many arguments as you want
    return k(*args) * np.eye(m)

# Testing for gaussian kernel
gauss_sep_ker = K(k_gauss, x, z)
print(gauss_sep_ker)

# Testing for approximation with RFF (random fourier features)
print("\n", "matrix kernel")
matrix_ker = K(K_approx2, x, z, w)
print(matrix_ker)

[[0.69925554 0.         0.         0.         0.        ]
 [0.         0.69925554 0.         0.         0.        ]
 [0.         0.         0.69925554 0.         0.        ]
 [0.         0.         0.         0.69925554 0.        ]
 [0.         0.         0.         0.         0.69925554]]

 matrix kernel
[[0.12723974 0.         0.         0.         0.        ]
 [0.         0.12723974 0.         0.         0.        ]
 [0.         0.         0.12723974 0.         0.        ]
 [0.         0.         0.         0.12723974 0.        ]
 [0.         0.         0.         0.         0.12723974]]


### Approximate function f

In [198]:
# create a list of all the w_1...w_d
d = 5
def list_d(d, n):
    list_w = []
    for i in range(d):
        w = np.random.normal(size = n) # should be N or d times
        list_w.append(w)
    return np.array(list_w)

def psi(x, d, list_w):
    """
    Args:
        list_w (list): list of size d of all w, every w is of size n

    Returns:
        list: a list of cos and sin from 1 to 2d
    """
    new_list = []
    for w in list_w:
        new_list.append(phi_cos(x,w))
        new_list.append(phi_sin(x,w))
    return (1/np.sqrt(d)) * np.array(new_list)

list_w = list_d(d,n)
print("psi list:", psi(x, d, list_w))

psi list: [ 0.17786327 -0.0914585   0.19438608  0.04705372  0.19451018  0.04653805
 -0.1649741   0.11306435  0.19254521  0.05409568]


### Phi

In [199]:
def phi(x, d, list_w): 
    psi_vector = []
    for i in range(len(x)):
        psi_vector.append(psi(x[i], d, list_w))
    return np.array(np.transpose(psi_vector))

phi_ = phi(x_vector, d, list_w)
print(phi_)

[[ 0.04284922 -0.02086076]
 [ 0.19535594 -0.1989091 ]
 [-0.1651649   0.16141143]
 [ 0.11278544  0.11809467]
 [-0.05092775  0.17686283]
 [ 0.19340725 -0.09337847]
 [-0.16573194 -0.10474188]
 [-0.11195055  0.1703794 ]
 [ 0.1987152   0.13957061]
 [-0.02263338 -0.1432482 ]]


### Alpha

In [201]:
def alpha(phi, y, lamba):
    N = len(y)
    print(lamba * N * np.identity(N))
    print(np.transpose(phi)@phi + lamba * N * np.identity(N))
    print(len(y), len(np.transpose(phi)))
    alpha = (np.transpose(phi) @ y)/(np.transpose(phi) @ phi + lamba * N * np.identity(N))
    return alpha

alpha_ = alpha(phi_, y, lam)
print(alpha_)

[[2.e-06 0.e+00]
 [0.e+00 2.e-06]]
[[ 0.200002   -0.05089738]
 [-0.05089738  0.200002  ]]
2 2


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 10)